In [2968]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

## Read the data

In [2969]:
# rfr file, rm file
file_name_rfr = r"..\Sray\original dirty data\New_rfr.xlsx"
file_name_rm = r"..\Sray\original dirty data\New_rm.xlsx"

#get the sorted fin file
# file_name_str = r"..\Sray\Analysis\AU\AU_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\CN\CN_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\HK\HK_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis\MY\MY_Bloomberg_fin_data_portfolio.xlsx"
file_name_str = r"..\Sray\Analysis\SG\SG_Bloomberg_fin_data_portfolio.xlsx"



indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'historical_market_cap'
# indicator = 'roa'
# indicator = 'vol_180d'
## indicator = 'return_com_eqy'

file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)
rfr_file = pd.ExcelFile(file_name_rfr)
rm_file = pd.ExcelFile(file_name_rm)

findata_sheet = pd.read_excel(
    fin_file, sheet_name="main", index_col=0, header=0)

porfolio_selection = pd.DataFrame(findata_sheet)


porfolio_selection.head()


,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2017,2017-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,77.471831,NaN,NaN,NaN,-0.6023,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2018,2018-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,88.120879,NaN,NaN,NaN,-6.3187,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2019,2019-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,29.957895,NaN,NaN,NaN,-5.9765,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2020,2020-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,-134.845070,NaN,NaN,NaN,-33.5734,NaN,NaN,NaN
59,EZRA HOLDINGS LTD,EZRA SP Equity,1,2021,2021-12-31,27.190422,0.0,0.846433,80.523781,0.0,...,NaN,NaN,-363.140845,NaN,NaN,NaN,-6.8196,NaN,NaN,NaN


## Sorting the data by firm name and then date

In [2970]:
portfolio_selection_sorted = porfolio_selection.sort_values(
    by=['firm_name', 'date'])
portfolio_selection_sorted.head(10)

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
4,AEM HOLDINGS LTD,AEM SP Equity,8,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-12.146377,7.806869,NaN,NaN,0.703200,4.012265,NaN,NaN
4,AEM HOLDINGS LTD,AEM SP Equity,8,2018,2018-12-31,41.625591,19.601330,24.607014,80.523781,482.330191,...,NaN,NaN,71.878555,221.110924,NaN,NaN,2.760650,1.450771,NaN,NaN
4,AEM HOLDINGS LTD,AEM SP Equity,8,2019,2019-12-31,NaN,NaN,NaN,NaN,NaN,...,-0.800330,30.824737,60.715333,95.638409,-0.312538,-0.090824,-0.351300,12.194501,NaN,NaN
4,AEM HOLDINGS LTD,AEM SP Equity,8,2020,2020-12-31,44.381851,34.913921,16.989117,81.095726,593.156683,...,-0.772134,9.850877,63.316760,70.446577,-0.758956,-0.538899,11.809200,13.248795,1.723032,21.921476
4,AEM HOLDINGS LTD,AEM SP Equity,8,2021,2021-12-31,44.804344,36.182423,16.989117,81.095726,614.707400,...,-0.838960,13.594900,-34.042491,113.952962,0.418044,-0.096504,7.532840,6.473027,1.552343,22.353920
4,AEM HOLDINGS LTD,AEM SP Equity,8,2022,2022-12-30,53.847702,43.823620,36.517532,81.095726,1600.330455,...,-0.752555,-18.978947,46.266254,101.846168,-0.640625,-0.018974,6.473700,7.180211,1.428175,26.914364
4,AEM HOLDINGS LTD,AEM SP Equity,8,2023,2023-01-31,NaN,NaN,NaN,NaN,NaN,...,-0.840080,-18.759538,45.768670,97.622491,-0.660997,-0.013433,6.473700,7.180211,1.428175,NaN
74,AIMS APAC REIT,AAREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN
74,AIMS APAC REIT,AAREIT SP Equity,11,2018,2018-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,20.764662,-18.426637,299.862490,0.130698,0.130698,4.767879,1.755459,NaN,NaN


## Ranking stocks percentile
| Country/Region | Percentile chosen (Top(%))|
| --- | --- |
| Australia (AU)| 20 |
| China (CN)| 20 |
| Malaysia (SG)| 20 |
| Hong Kong (SG)| 25 |
| Singapore (SG)| 25 |

## Excess Return
At the end of each month, compute the cross-sectional portfolio return of esg_bottom20. For example, at the end of August, after selecting the bottom 20% stocks based on August ESG scores, compute the raw returns of each stocks in the high-rated portfolio (the bottom 20%) by the following equation: $$ret_{i}  = P_{1}/P_{0} -1 $$(for example, after ranking stocks at the end of Aug, calculate the individual returns by using $$(Sept-price/Aug-price) – 1)$$ Next, compute excess returns of each stocks: $$r_{i} = ret_{i} – r_{f}$$ where r_i is the excess return of stock i and r_f is the respective country’s monthly risk-free rate. So now we have r_1, …, r_n where n = number of stocks in the high-rated portfolio. Next, we equal-weight (average) all the r in the esg_bottom 20 and find the portfolio mean (denoted as μ_t) for each month. This generates the High portfolio excess return (equal weight) in the Excel file for each month. Scroll down the Excel file, compute the mean of all the months’ excess returns, and test for significance (H0: mean > 0). Indicate the t-stat value.

In [2971]:
"""
change data format
"""

portfolio_selection_sorted['date'] = portfolio_selection_sorted['date'].astype(
    'datetime64')
portfolio_selection_sorted.dtypes


firm_name                        object
tkr                              object
sec_no                            int64
year                              int64
date                     datetime64[ns]
esg                             float64
e                               float64
s                               float64
g                               float64
es                              float64
eg                              float64
sg                              float64
esg_cubic                       float64
px                              float64
fcf                             float64
pe                              float64
beta                            float64
roa                             float64
vol_180d                        float64
sales                           float64
ret                             float64
historical_market_cap           float64
ep                              float64
ep_sec_mean                     float64
ep_sec_sd                       float64


In [2972]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
quantile_rank = 0
quantile_path = ''
quantile_folder = ''


quantile_folder = "..\Sray\Analysis\\" + file_name[:2] + "\\" + indicator
quantile_path = quantile_folder + "\\" + file_name[:2] +"_portfolio_" + indicator + "_quantile_rank.xlsx"
bottom_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_bottom20.xlsx"
reporting_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting.xlsx"
rfr_sheet = pd.read_excel(rfr_file, sheet_name= file_name[:2] )
rm_sheet = pd.read_excel(rm_file, sheet_name= file_name[:2] )

try:
    os.makedirs(quantile_folder)
except FileExistsError:
    # directory already exists
    pass

if file_name[:2].lower() == 'AU'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'CN'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'HK'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'MY'.lower():
    quantile_rank = 4

elif file_name[:2].lower() == 'SG'.lower():
    quantile_rank = 4
    

In [2973]:
rfr_sheet = rfr_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rfr_yr': 'rfr'}, axis=1)
rfr_sheet

,year,rfr
0,2017,0.01762
1,2018,0.02013
2,2019,0.01657
3,2020,0.00305
4,2021,0.00442


In [2974]:
rm_sheet = rm_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rm_yr': 'rm'}, axis=1)
rm_sheet

,year,rm
0,2017,0.116881
1,2018,-0.131644
2,2019,0.010238
3,2020,-0.098272
4,2021,0.108782


In [2975]:
rfr_rm_sheet = pd.merge(rfr_sheet, rm_sheet, how='left', on='year')
rfr_rm_sheet

,year,rfr,rm
0,2017,0.01762,0.116881
1,2018,0.02013,-0.131644
2,2019,0.01657,0.010238
3,2020,0.00305,-0.098272
4,2021,0.00442,0.108782


In [2976]:
"""
gets all the companies, dropping the empty columns
"""
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates()

#set the data to a list
companies = companies.tolist()


#ensure the number of companies exist
len(companies)


105

In [2977]:
# new_companies_data.to_excel("SG_data_2.28_esgj3mean.xlsx")

### Rank the companies 

In [2978]:
companies_sray = []
all_quantile_rank = []

initial_year = 2017
stop_year = 2023


for year in range(initial_year, stop_year+1, 1):

        # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
        temp_dataframe = portfolio_selection_sorted.loc[(
            portfolio_selection_sorted['year'] == year)].copy()

        
        """
        sort by the relative value and then divide them to relative division
        """

        temp_dataframe.sort_values(by=[indicator], inplace=True)

        temp_dataframe['quantile_rank'] = pd.qcut(temp_dataframe[indicator],
                                                    q=quantile_rank, labels=False, duplicates='drop')

        temp_dataframe_quantile = temp_dataframe.copy()

        companies_sray.append(temp_dataframe)

        all_quantile_rank.append(temp_dataframe_quantile)


In [2979]:
all_sort_companies_data = pd.concat(all_quantile_rank)

all_sort_companies_data.to_excel(
    quantile_path, sheet_name="main")


In [2980]:
sort_companies_data = pd.concat(companies_sray)
sort_companies_data.head()

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
firm no,,,,,,,,,,,,,,,,,,,,,
98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,0.000000,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,1936.849257,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,205.008084,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


In [2981]:
all_sort_companies_data = all_sort_companies_data.reset_index()
all_sort_companies_data = pd.merge(all_sort_companies_data, rfr_rm_sheet, how='left', on='year')
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('ret')+1, 'rfr', all_sort_companies_data.pop('rfr'))
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('rfr')+1, 'rm', all_sort_companies_data.pop('rm'))

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


In [2982]:
df_sec_company_count_sum = []
df_sec_company_count = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].count())
df_sec_company_count = df_sec_company_count.rename({'tkr':'tkr_count'}, axis=1)
df_sec_company_nunique = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].nunique())
df_sec_company_nunique = df_sec_company_nunique.rename({'tkr':'tkr_nunique'}, axis=1)

df_sec_company_count = pd.merge(df_sec_company_count, df_sec_company_nunique, on=['quantile_rank','year'])

In [2983]:
df_sec_company_count_sum = list()
# for rank in range (0, quantile_rank, 1):
groups = df_sec_company_count.groupby(level='quantile_rank')

    # loop through the groups to create separate dataframes
dfs = {}
for name, group in groups:
    dfs[name] = group.copy()
        
    # display the separate dataframes
for name, df in dfs.items():

        # Create a new row with the sum of all values above it
    sum_row = df.sum()
    sum_index = (name, 'Sum')
    sum_row.name = sum_index

        # Append the new row to the original DataFrame
    temp_1 = df.append(sum_row)
    df_sec_company_count_sum.append(temp_1)
# df_sec_company_count_all = pd.concat(df_sec_company_count_sum)
df_sec_company_count_sum_pd = pd.concat(df_sec_company_count_sum)

In [2984]:
book = load_workbook(quantile_path)
writer = pd.ExcelWriter(quantile_path, engine = 'openpyxl')
writer.book = book
df_sec_company_count_sum_pd.to_excel(writer, sheet_name='quantile sec no', index=True)
writer.close()

In [2985]:
# sort_companies_data['date'] = sort_companies_data['date'].dt.strftime(
#     '%Y-%m')

all_sort_companies_data = all_sort_companies_data.sort_values(
    by=["date", "quantile_rank"])

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


**SRAY- Ret already calculated using excel by Dr May**
<br>**BLOOMBERG- Ret calculated using PYTHON CODE 2_portfolio.ipynb**

In [2986]:
sort_companies_data_bottom20per = all_sort_companies_data.query("quantile_rank == 0")

sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,270.840176,345.933328,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,515.451977,270.840176,345.933328,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,28.119766,48.408147,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0


In [2987]:
sort_companies_data_bottom20per.columns

Index(['firm no', 'firm_name', 'tkr', 'sec_no', 'year', 'date', 'esg', 'e',
       's', 'g', 'es', 'eg', 'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta',
       'roa', 'vol_180d', 'sales', 'ret', 'rfr', 'rm', 'historical_market_cap',
       'ep', 'ep_sec_mean', 'ep_sec_sd', 'ep_z', 'sp', 'sp_sec_mean',
       'sp_sec_sd', 'sp_z', 'cfp', 'cfp_sec_mean', 'cfp_sec_sd', 'cfp_z',
       'value', 'roa_sec_mean', 'roa_sec_sd', 'roa_z', 'esg_value',
       'quantile_rank'],
      dtype='object')

In [2988]:
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates().tolist()
print(companies)

['AEM HOLDINGS LTD', 'AIMS APAC REIT', 'APAC REALTY LTD', 'ARA US HOSPITALITY TRUST', 'ASPEN GROUP HOLDINGS LTD', 'BANYAN TREE HOLDINGS LTD', 'BUKIT SEMBAWANG ESTATES LTD', 'BUMITAMA AGRI LTD', 'BUND CENTER INVESTMENT LTD', 'CAPITALAND ASCENDAS REIT', 'CAPITALAND ASCOTT TRUST', 'CAPITALAND CHINA TRUST', 'CAPITALAND INDIA TRUST', 'CAPITALAND INTEGRATED COMMER', 'CAPITALAND INVESTMENT LTD/SI', 'CDL HOSPITALITY TRUSTS', 'CENTURION CORP LTD', 'CHINA FISHERY GROUP LTD', 'CHINA SUNSINE CHEMICAL HLDGS', 'CHIP ENG SENG CORP LTD', 'CITY DEVELOPMENTS LTD', 'COMFORTDELGRO CORP LTD', 'COSCO SHIPPING INTERNATIONAL', 'CROMWELL REIT EUR', 'DBS GROUP HOLDINGS LTD', 'DEL MONTE PACIFIC LTD', 'DELFI LTD', 'DFI RETAIL GROUP HOLDINGS LT', 'EC WORLD REIT', 'ESR-LOGOS REIT', 'EZRA HOLDINGS LTD', 'FAR EAST HOSPITALITY TRUST', 'FIRST RESOURCES LTD', 'FIRST SPONSOR GROUP LTD', 'FRASER AND NEAVE LTD', 'FRASERS CENTREPOINT TRUST', 'FRASERS LOGISTICS & COMMERCI', 'FRASERS PROPERTY LTD', 'GENTING SINGAPORE LTD', 'G

In [2989]:
companies_sec=[]
initial_year=2017
stop_year=2023

company_dict1 = {}
company_dict_all = {}
sum_esg_sray = {}
sum_ret_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =0 and save it into company dictionary 
for year in range(initial_year,stop_year+1,1):    

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        each_year_companies_all_quantile = all_sort_companies_data.loc[(
            all_sort_companies_data['year'] == year)]

        sum = each_year_companies[indicator].sum()
        sum_esg_sray.setdefault(year, sum)

        sum_ret = each_year_companies['ret'].sum()
        sum_ret_dict.setdefault(year, sum_ret)

        companies = each_year_companies.iloc[:, 0]
        companies= companies.tolist() 

        total_companies = (each_year_companies_all_quantile.iloc[:, 0]).tolist()


        for c in range (len(companies)):
            company_dict1.setdefault(year,[]).append(companies[c])
            #company_dict2.setdefault(start_date,[]).append(companies[c])
        for c_all in range(len(total_companies)):
            company_dict_all.setdefault(
                year, []).append(total_companies[c_all])    
        

#make sure the list is correct     
#print(company_dict)
# print(f"company_dict1: {company_dict1}")
print(len(company_dict1))
print(f"sum_esg_sray_dic: {sum_esg_sray}")
print(f"sum_ret_dict: {sum_ret_dict}")
# print(len(company_dict2))


7
sum_esg_sray_dic: {2017: 299.12699999999995, 2018: 365.339, 2019: 336.511, 2020: 507.745, 2021: 381.75700000000006, 2022: 447.38899999999995, 2023: 448.25700000000006}
sum_ret_dict: {2017: 0.0, 2018: -1.0152306794382864, 2019: 0.5133517996271684, 2020: -1.981120663968954, 2021: -0.2698670635088647, 2022: -1.2716860176580207, 2023: 0.8732419418662392}


If it is equal-weighted, compute the mean of the stocks returns in the portfolio. 
$μ_(p,t)=  (\sum ret_{(i,t)} )/n$; i = 1, 2, 3, …, n
Where $ret_(i,t) = return-of-stock-i-at-time-t$, n = number of stocks in the portfolio


In [2990]:
no_of_companies = {}
equal_weight_dict = {}
for date_range, company_names in company_dict1.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    #equal weightage
    # equal_weight = np.repeat(1/len(v), len(v))
    equal_weight = 1/len(company_names)
        # print(f"factor_weight: {factor_weight}")

    equal_weight_dict.setdefault(date_range, float(equal_weight))
    no_of_companies.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"equal_weight: {equal_weight_dict}")
print(f"no_of_companies: {no_of_companies}")


equal_weight: {2017: 0.041666666666666664, 2018: 0.04, 2019: 0.04, 2020: 0.04, 2021: 0.038461538461538464, 2022: 0.038461538461538464, 2023: 0.038461538461538464}
no_of_companies: {2017: 24.0, 2018: 25.0, 2019: 25.0, 2020: 25.0, 2021: 26.0, 2022: 26.0, 2023: 26.0}


### total companies from all quantile

In [2991]:
no_of_companies_all = {}
for date_range, company_names in company_dict_all.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    no_of_companies_all.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"no_of_companies: {no_of_companies_all}")


no_of_companies: {2017: 105.0, 2018: 105.0, 2019: 105.0, 2020: 105.0, 2021: 105.0, 2022: 105.0, 2023: 105.0}


## Calculate the number of companies (Bottom 20%) based on the date

In [2992]:
no_of_companies_items = no_of_companies.items()
no_of_companies_list = list(no_of_companies_items)

no_of_companies_df = pd.DataFrame(no_of_companies_list)
no_of_companies_df.columns = ['year', 'no of firms(bottom 20)']
no_of_companies_df.head()

,year,no of firms(bottom 20)
0,2017,24.0
1,2018,25.0
2,2019,25.0
3,2020,25.0
4,2021,26.0


## Calculate the number of companies based on the date

In [2993]:
no_of_companies_all_items = no_of_companies_all.items()
no_of_companies_all_list = list(no_of_companies_all_items)

no_of_companies_all_df = pd.DataFrame(no_of_companies_all_list)
no_of_companies_all_df.columns = ['year', 'no of firms']
no_of_companies_all_df.head()

,year,no of firms
0,2017,105.0
1,2018,105.0
2,2019,105.0
3,2020,105.0
4,2021,105.0


### Make the equal weight item into dataframe
Purpose: use for merging later on

In [2994]:
equal_weight_items = equal_weight_dict.items()
equal_weight_list = list(equal_weight_items)

equal_weight_df = pd.DataFrame(equal_weight_list)
equal_weight_df.columns = ['year', 'eq_wg']
equal_weight_df.head()


,year,eq_wg
0,2017,0.041667
1,2018,0.040000
2,2019,0.040000
3,2020,0.040000
4,2021,0.038462


In [2995]:
# no_of_companies_df.to_excel("eq_wg_no_of_companies_bottom20.xlsx")

In [2996]:
sum_ret_items = sum_ret_dict.items()
sum_ret_list = list(sum_ret_items)

sum_ret_df = pd.DataFrame(sum_ret_list)
sum_ret_df.columns = ['year', 'sum_ret']
sum_ret_df.replace(0, np.nan, inplace=True)
sum_ret_df.head()


,year,sum_ret
0,2017,NaN
1,2018,-1.015231
2,2019,0.513352
3,2020,-1.981121
4,2021,-0.269867


### Make the sum of the esg_sray into dataframe
Purpose: merge into complete dataframe to make the data to be able to vefiry

In [2997]:
sum_esg_sray_dic_items = sum_esg_sray.items()
sum_esg_sray_list = list(sum_esg_sray_dic_items)

sum_esg_sray_df = pd.DataFrame(sum_esg_sray_list)
sum_esg_sray_df.columns = ['year', 'sum_'+indicator]
sum_esg_sray_df.head()


,year,sum_vol_180d
0,2017,299.127
1,2018,365.339
2,2019,336.511
3,2020,507.745
4,2021,381.757


### Merging with the data of top 20% **only**

In [2998]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    equal_weight_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    sum_esg_sray_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    sum_ret_df, on='year')
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,NaN,11.891550,0.846902,NaN,NaN,0.0,0.041667,299.127,NaN
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0,0.041667,299.127,NaN
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,0.707107,0.235702,11.891550,0.846902,0.707107,29.782875,0.0,0.041667,299.127,NaN
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0,0.041667,299.127,NaN
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,NaN,5.630938,3.148304,NaN,NaN,0.0,0.041667,299.127,NaN


### Calculate the factor weight

In [2999]:
if sort_companies_data_bottom20per[indicator].empty and sort_companies_data_bottom20per['sum_'+indicator].empty:
    sort_companies_data_bottom20per['fc_wg'] = None
else:
    sort_companies_data_bottom20per['fc_wg'] = sort_companies_data_bottom20per[indicator] / \
        sort_companies_data_bottom20per['sum_'+indicator]

sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,11.891550,0.846902,NaN,NaN,0.0,0.041667,299.127,NaN,0.032337
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,5.630938,3.148304,NaN,NaN,0.0,0.041667,299.127,NaN,0.032615
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,0.235702,11.891550,0.846902,0.707107,29.782875,0.0,0.041667,299.127,NaN,0.033414
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,5.630938,3.148304,NaN,NaN,0.0,0.041667,299.127,NaN,0.034129
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,5.630938,3.148304,NaN,NaN,0.0,0.041667,299.127,NaN,0.037135


In [3000]:
sort_companies_data_bottom20per['r'] = (
    sort_companies_data_bottom20per['ret'] - sort_companies_data_bottom20per['rfr'])

"""
r = ret -rfr
"""
sort_companies_data_bottom20per['eq_wg_r'] = (
    sort_companies_data_bottom20per['r'] * sort_companies_data_bottom20per['eq_wg'])
sort_companies_data_bottom20per['fc_wg_r'] = (
    sort_companies_data_bottom20per['r'] * sort_companies_data_bottom20per['fc_wg'])

"""
ret
"""
sort_companies_data_bottom20per['eq_wg_ret'] = (
    sort_companies_data_bottom20per['ret'] * sort_companies_data_bottom20per['eq_wg'])
sort_companies_data_bottom20per['fc_wg_ret'] = (
    sort_companies_data_bottom20per['ret'] * sort_companies_data_bottom20per['fc_wg'])

sort_companies_data_bottom20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,0.0,0.041667,299.127,NaN,0.032337,NaN,NaN,NaN,NaN,NaN
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.032615,NaN,NaN,NaN,NaN,NaN
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,0.0,0.041667,299.127,NaN,0.033414,NaN,NaN,NaN,NaN,NaN
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.034129,NaN,NaN,NaN,NaN,NaN
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,0.0,0.041667,299.127,NaN,0.037135,NaN,NaN,NaN,NaN,NaN
5,90,OUE COMMERCIAL REAL ESTATE I,OUECT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.037389,NaN,NaN,NaN,NaN,NaN
6,46,STARHILL GLOBAL REIT,SGREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.037870,NaN,NaN,NaN,NaN,NaN
7,88,PARAGON REIT,PGNREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.037880,NaN,NaN,NaN,NaN,NaN
8,95,EC WORLD REIT,ECWREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.040023,NaN,NaN,NaN,NaN,NaN
9,31,CAPITALAND INTEGRATED COMMER,CICT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.0,0.041667,299.127,NaN,0.040618,NaN,NaN,NaN,NaN,NaN


## Excess return portfolio
### Equal Weight and Factor Weight

In [3001]:
initial_year = 2017
stop_year = 2023

lo_eq_wg_r_p_dict = {}
lo_fc_wg_r_p_dict = {}
eq_wg_ret_p_dict = {}
fc_wg_ret_p_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        """
        r
        """
        lo_eq_wg_r_p = each_year_companies['eq_wg_r'].sum()
        lo_fc_wg_r_p = each_year_companies['fc_wg_r'].sum()


        """
        ret
        """
        eq_wg_ret_p = each_year_companies['eq_wg_ret'].sum()
        fc_wg_ret_p = each_year_companies['fc_wg_ret'].sum()


        lo_eq_wg_r_p_dict.setdefault(year, lo_eq_wg_r_p)
        lo_fc_wg_r_p_dict.setdefault(year, lo_fc_wg_r_p)

        eq_wg_ret_p_dict.setdefault(year, eq_wg_ret_p)
        fc_wg_ret_p_dict.setdefault(year, fc_wg_ret_p)


print(f"lo_eq_wg_r_p_dict: {lo_eq_wg_r_p_dict}")
print(f"lo_fc_wg_r_p_dict: {lo_fc_wg_r_p_dict}")
print(f"eq_wg_ret_p_dict: {eq_wg_ret_p_dict}")
print(f"fc_wg_ret_p_dict: {fc_wg_ret_p_dict}")
# print(len(company_dict2))


lo_eq_wg_r_p_dict: {2017: 0.0, 2018: -0.060739227177531455, 2019: 0.004626871985086732, 2020: -0.08229482655875817, 2021: -0.014799502442648644, 2022: 0.0, 2023: 0.0}
lo_fc_wg_r_p_dict: {2017: 0.0, 2018: -0.06363828657218715, 2019: 0.01123512164419101, 2020: -0.0811813291287683, 2021: -0.014005161175997803, 2022: 0.0, 2023: 0.0}
eq_wg_ret_p_dict: {2017: 0.0, 2018: -0.04060922717753145, 2019: 0.020534071985086735, 2020: -0.07924482655875817, 2021: -0.010379502442648642, 2022: -0.048911000679154645, 2023: 0.03358622853331689}
fc_wg_ret_p_dict: {2017: 0.0, 2018: -0.043508286572187145, 2019: 0.02717459589614711, 2020: -0.0781313291287683, 2021: -0.009585161175997805, 2022: -0.0493540076018668, 2023: 0.03252771212346206}


In [3002]:
lo_eq_wg_r_p_dict_items = lo_eq_wg_r_p_dict.items()
lo_eq_wg_r_p_dict_list = list(lo_eq_wg_r_p_dict_items)

lo_eq_wg_r_p_dict_df = pd.DataFrame(lo_eq_wg_r_p_dict_list)
lo_eq_wg_r_p_dict_df.columns = ['year', 'lo_eq_wg_r_p']
lo_eq_wg_r_p_dict_df.replace(0, np.nan, inplace = True)
lo_eq_wg_r_p_dict_df.head()

,year,lo_eq_wg_r_p
0,2017,NaN
1,2018,-0.060739
2,2019,0.004627
3,2020,-0.082295
4,2021,-0.014800


In [3003]:
lo_fc_wg_r_p_dict_items = lo_fc_wg_r_p_dict.items()
lo_fc_wg_r_p_dict_list = list(lo_fc_wg_r_p_dict_items)

lo_fc_wg_r_p_dict_df = pd.DataFrame(lo_fc_wg_r_p_dict_list)
lo_fc_wg_r_p_dict_df.columns = ['year', 'lo_fc_wg_r_p']
lo_fc_wg_r_p_dict_df.replace(0, np.nan, inplace=True)
lo_fc_wg_r_p_dict_df.head()


,year,lo_fc_wg_r_p
0,2017,NaN
1,2018,-0.063638
2,2019,0.011235
3,2020,-0.081181
4,2021,-0.014005


In [3004]:
eq_wg_ret_p_dict_items = eq_wg_ret_p_dict.items()
eq_wg_ret_p_dict_list = list(eq_wg_ret_p_dict_items)

eq_wg_ret_p_dict_df = pd.DataFrame(eq_wg_ret_p_dict_list)
eq_wg_ret_p_dict_df.columns = ['year', 'eq_wg_ret_p']
eq_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_ret_p_dict_df.head()


,year,eq_wg_ret_p
0,2017,NaN
1,2018,-0.040609
2,2019,0.020534
3,2020,-0.079245
4,2021,-0.010380


In [3005]:
fc_wg_ret_p_dict_items = fc_wg_ret_p_dict.items()
fc_wg_ret_p_dict_list = list(fc_wg_ret_p_dict_items)

fc_wg_ret_p_dict_df = pd.DataFrame(fc_wg_ret_p_dict_list)
fc_wg_ret_p_dict_df.columns = ['year', 'fc_wg_ret_p']
fc_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_ret_p_dict_df.head()


,year,fc_wg_ret_p
0,2017,NaN
1,2018,-0.043508
2,2019,0.027175
3,2020,-0.078131
4,2021,-0.009585


In [3006]:
SG_esg_int = pd.merge(no_of_companies_all_df, no_of_companies_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_eq_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_fc_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,2017,105.0,24.0,NaN,NaN,NaN,NaN
1,2018,105.0,25.0,-0.060739,-0.063638,-0.040609,-0.043508
2,2019,105.0,25.0,0.004627,0.011235,0.020534,0.027175
3,2020,105.0,25.0,-0.082295,-0.081181,-0.079245,-0.078131
4,2021,105.0,26.0,-0.014800,-0.014005,-0.010380,-0.009585


### Merging the eq and fc wg of the r_p into the big dataframe

In [3007]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_eq_wg_r_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_fc_wg_r_p_dict_df, on='year')

sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    eq_wg_ret_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    fc_wg_ret_p_dict_df, on='year')

sort_companies_data_bottom20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret,lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,0.032337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.032615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,0.033414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.034129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,0.037135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,90,OUE COMMERCIAL REAL ESTATE I,OUECT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.037389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,46,STARHILL GLOBAL REIT,SGREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.037870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,88,PARAGON REIT,PGNREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.037880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,95,EC WORLD REIT,ECWREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.040023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,31,CAPITALAND INTEGRATED COMMER,CICT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.040618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3008]:
sort_companies_data_bottom20per['rm_r_y1'] = (
    sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr']) * 12
sort_companies_data_bottom20per['rm_r_y2'] = (((1+(
    sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr'])) ** 12) - 1)

sort_companies_data_bottom20per['lo_eq_wg_r_p_y1'] = sort_companies_data_bottom20per['lo_eq_wg_r_p'] *12
sort_companies_data_bottom20per['lo_eq_wg_r_p_y2'] = (((1 + sort_companies_data_bottom20per['lo_eq_wg_r_p'])**12) - 1)
sort_companies_data_bottom20per['eq_r_alpha_y1'] = sort_companies_data_bottom20per['lo_eq_wg_r_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['eq_r_alpha_y2'] = sort_companies_data_bottom20per['lo_eq_wg_r_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,NaN,NaN,NaN,1.191128,2.113208,NaN,NaN,NaN,NaN


In [3009]:
sort_companies_data_bottom20per['lo_fc_wg_r_p_y1'] = sort_companies_data_bottom20per['lo_fc_wg_r_p'] * 12
sort_companies_data_bottom20per['lo_fc_wg_r_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_fc_wg_r_p'])**12) - 1)
sort_companies_data_bottom20per['fc_r_alpha_y1'] = sort_companies_data_bottom20per['lo_fc_wg_r_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['fc_r_alpha_y2'] = sort_companies_data_bottom20per['lo_fc_wg_r_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3010]:

# sort_companies_data_bottom20per.to_excel(
#     "../3.04/SG_esg_bottom20.xlsx", sheet_name="main")

## CAPM-adjusted return ($r_{c}$)

$r_(c,t)=r_(f,t)+β_(p,t) (r_(m,t)-r_(f,t))$

In [3011]:
sort_companies_data_bottom20per['rc'] = sort_companies_data_bottom20per['rfr'] + \
    (sort_companies_data_bottom20per['beta'] *
    (sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr']))
    
"""
equal weight rc
"""
sort_companies_data_bottom20per['lo_eq_wg_rc_int'] = sort_companies_data_bottom20per ['eq_wg'] * sort_companies_data_bottom20per['rc']

"""
factor weight rc
"""
sort_companies_data_bottom20per['lo_fc_wg_rc_int'] = sort_companies_data_bottom20per['fc_wg'] * \
    sort_companies_data_bottom20per['rc']

sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000704,-0.000029,-0.000023
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033105,0.001379,0.001080
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.092750,0.003865,0.003099
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016707,0.000696,0.000570
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046306,0.001929,0.001720


In [3012]:
initial_year = 2017
stop_year = 2023

lo_eq_wg_rc_p_dict = {}
lo_fc_wg_rc_p_dict = {}

# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        """
        rc
        """
        lo_eq_wg_rc_p = each_year_companies['lo_eq_wg_rc_int'].sum()
        lo_fc_wg_rc_p = each_year_companies['lo_fc_wg_rc_int'].sum()

        lo_eq_wg_rc_p_dict.setdefault(year, lo_eq_wg_rc_p)
        lo_fc_wg_rc_p_dict.setdefault(year, lo_fc_wg_rc_p)


print(f"lo_eq_wg_rc_p_dict: {lo_eq_wg_rc_p_dict}")
print(f"lo_fc_wg_rc_p_dict: {lo_fc_wg_rc_p_dict}")


lo_eq_wg_rc_p_dict: {2017: 0.05425172827433122, 2018: -0.06122578431338142, 2019: 0.012871736656351896, 2020: -0.06643462689091102, 2021: 0.0780497518520324, 2022: 0.0, 2023: 0.0}
lo_fc_wg_rc_p_dict: {2017: 0.05560643470624652, 2018: -0.06450353262031243, 2019: 0.01286091996478149, 2020: -0.06690020078027402, 2021: 0.07950592158093774, 2022: 0.0, 2023: 0.0}


In [3013]:
lo_eq_wg_rc_p_dict_items = lo_eq_wg_rc_p_dict.items()
lo_eq_wg_rc_p_dict_list = list(lo_eq_wg_rc_p_dict_items)

lo_eq_wg_rc_p_dict_df = pd.DataFrame(lo_eq_wg_rc_p_dict_list)
lo_eq_wg_rc_p_dict_df.columns = ['year', 'lo_eq_wg_rc_p']
lo_eq_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
lo_eq_wg_rc_p_dict_df.head()


,year,lo_eq_wg_rc_p
0,2017,0.054252
1,2018,-0.061226
2,2019,0.012872
3,2020,-0.066435
4,2021,0.078050


In [3014]:
lo_fc_wg_rc_p_dict_items = lo_fc_wg_rc_p_dict.items()
lo_fc_wg_rc_p_dict_list = list(lo_fc_wg_rc_p_dict_items)

lo_fc_wg_rc_p_dict_df = pd.DataFrame(lo_fc_wg_rc_p_dict_list).dropna()
lo_fc_wg_rc_p_dict_df.columns = ['year', 'lo_fc_wg_rc_p']
lo_fc_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
lo_fc_wg_rc_p_dict_df.head()


,year,lo_fc_wg_rc_p
0,2017,0.055606
1,2018,-0.064504
2,2019,0.012861
3,2020,-0.066900
4,2021,0.079506


In [3015]:
SG_esg_int = pd.merge(SG_esg_int, lo_eq_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_fc_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()

,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_rc_p,lo_fc_wg_rc_p
0,2017,105.0,24.0,NaN,NaN,NaN,NaN,0.054252,0.055606
1,2018,105.0,25.0,-0.060739,-0.063638,-0.040609,-0.043508,-0.061226,-0.064504
2,2019,105.0,25.0,0.004627,0.011235,0.020534,0.027175,0.012872,0.012861
3,2020,105.0,25.0,-0.082295,-0.081181,-0.079245,-0.078131,-0.066435,-0.066900
4,2021,105.0,26.0,-0.014800,-0.014005,-0.010380,-0.009585,0.078050,0.079506


In [3016]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_eq_wg_rc_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_fc_wg_rc_p_dict_df, on='year')

sort_companies_data_bottom20per.head(15)

,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int,lo_eq_wg_rc_p,lo_fc_wg_rc_p
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,NaN,NaN,NaN,NaN,-0.000704,-0.000029,-0.000023,0.054252,0.055606
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.033105,0.001379,0.001080,0.054252,0.055606
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,NaN,NaN,NaN,NaN,NaN,0.092750,0.003865,0.003099,0.054252,0.055606
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.016707,0.000696,0.000570,0.054252,0.055606
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,NaN,NaN,NaN,NaN,0.046306,0.001929,0.001720,0.054252,0.055606
5,90,OUE COMMERCIAL REAL ESTATE I,OUECT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.037244,0.001552,0.001393,0.054252,0.055606
6,46,STARHILL GLOBAL REIT,SGREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.046942,0.001956,0.001778,0.054252,0.055606
7,88,PARAGON REIT,PGNREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.045979,0.001916,0.001742,0.054252,0.055606
8,95,EC WORLD REIT,ECWREIT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.020449,0.000852,0.000818,0.054252,0.055606
9,31,CAPITALAND INTEGRATED COMMER,CICT SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.020469,0.000853,0.000831,0.054252,0.055606


In [3017]:
sort_companies_data_bottom20per['lo_eq_wg_rc_p_y1'] = sort_companies_data_bottom20per['lo_eq_wg_rc_p'] * 12
sort_companies_data_bottom20per['eq_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_eq_wg_rc_p'])**12) - 1)
sort_companies_data_bottom20per['eq_rc_alpha_y1'] = sort_companies_data_bottom20per['lo_eq_wg_rc_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['eq_rc_alpha_y2'] = sort_companies_data_bottom20per['eq_wg_rc_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int,lo_eq_wg_rc_p,lo_fc_wg_rc_p,lo_eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,NaN,-0.000704,-0.000029,-0.000023,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,0.033105,0.001379,0.001080,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,NaN,0.092750,0.003865,0.003099,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,NaN,0.016707,0.000696,0.000570,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,NaN,0.046306,0.001929,0.001720,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119


In [3018]:
sort_companies_data_bottom20per['lo_fc_wg_rc_p_y1'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p'] * 12
sort_companies_data_bottom20per['lo_fc_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_fc_wg_rc_p'])**12) - 1)
sort_companies_data_bottom20per['fc_rc_alpha_y1'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['fc_rc_alpha_y2'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,lo_eq_wg_rc_p,lo_fc_wg_rc_p,lo_eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2,lo_fc_wg_rc_p_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y1,fc_rc_alpha_y2
0,98,NETLINK NBN TRUST,NETLINK SP Equity,9,2017,2017-12-31,23.750126,0.000000,11.577993,59.542442,...,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119,0.667277,0.914363,-0.523851,-1.198845
1,91,FIRST SPONSOR GROUP LTD,FSG SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119,0.667277,0.914363,-0.523851,-1.198845
2,6,SINGAPORE TELECOMMUNICATIONS,ST SP Equity,9,2017,2017-12-31,59.330048,44.035034,43.984280,89.855507,...,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119,0.667277,0.914363,-0.523851,-1.198845
3,42,CAPITALAND ASCOTT TRUST,CLAS SP Equity,11,2017,2017-12-31,NaN,NaN,NaN,NaN,...,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119,0.667277,0.914363,-0.523851,-1.198845
4,44,MAPLETREE PAN ASIA COM TRUST,MPACT SP Equity,11,2017,2017-12-31,30.882204,12.352764,16.596130,63.576160,...,0.054252,0.055606,0.651021,0.885089,-0.540107,-1.228119,0.667277,0.914363,-0.523851,-1.198845


In [3019]:
SG_int_wanted = sort_companies_data_bottom20per[[
    'year', 'rm_r_y1', 'rm_r_y2', 'lo_eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_r_alpha_y2', 'lo_eq_wg_r_p_y2', 'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_r_alpha_y2', 'lo_fc_wg_r_p_y2', 'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_rc_alpha_y2', 'eq_wg_rc_p_y2', 'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_rc_alpha_y2', 'lo_fc_wg_rc_p_y2']]
SG_int_wanted = SG_int_wanted.drop_duplicates()
SG_int_wanted.head()


,year,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,eq_r_alpha_y1,eq_r_alpha_y2,lo_eq_wg_r_p_y2,lo_fc_wg_r_p_y1,fc_r_alpha_y1,fc_r_alpha_y2,lo_fc_wg_r_p_y2,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,lo_fc_wg_rc_p_y2
0,2017,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.651021,-0.540107,-1.228119,0.885089,0.667277,-0.523851,-1.198845,0.914363
24,2018,-1.821289,-0.861280,-0.728871,1.092419,0.332729,-0.528552,-0.763659,1.057630,0.315560,-0.545720,-0.734709,1.086580,0.329806,-0.531474,-0.774042,1.047247,0.310549,-0.550732
49,2019,-0.075984,-0.073393,0.055522,0.131507,0.130351,0.056957,0.134821,0.210806,0.216866,0.143473,0.154461,0.230445,0.239272,0.165879,0.154331,0.230315,0.239123,0.165729
74,2020,-1.215864,-0.722509,-0.987538,0.228327,0.079320,-0.643189,-0.974176,0.241689,0.084550,-0.637959,-0.797216,0.418649,0.160774,-0.561735,-0.802802,0.413062,0.158158,-0.564351
99,2021,1.252343,2.291070,-0.177594,-1.429937,-2.454898,-0.163828,-0.168062,-1.420405,-2.446772,-0.155702,0.936597,-0.315746,-0.826929,1.464142,0.954071,-0.298272,-0.786689,1.504381


In [3020]:
SG_esg_int = pd.merge(SG_esg_int, SG_int_wanted,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_rc_p,lo_fc_wg_rc_p,rm_r_y1,...,fc_r_alpha_y2,lo_fc_wg_r_p_y2,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,lo_fc_wg_rc_p_y2
0,2017,105.0,24.0,NaN,NaN,NaN,NaN,0.054252,0.055606,1.191128,...,NaN,NaN,0.651021,-0.540107,-1.228119,0.885089,0.667277,-0.523851,-1.198845,0.914363
1,2018,105.0,25.0,-0.060739,-0.063638,-0.040609,-0.043508,-0.061226,-0.064504,-1.821289,...,0.315560,-0.545720,-0.734709,1.086580,0.329806,-0.531474,-0.774042,1.047247,0.310549,-0.550732
2,2019,105.0,25.0,0.004627,0.011235,0.020534,0.027175,0.012872,0.012861,-0.075984,...,0.216866,0.143473,0.154461,0.230445,0.239272,0.165879,0.154331,0.230315,0.239123,0.165729
3,2020,105.0,25.0,-0.082295,-0.081181,-0.079245,-0.078131,-0.066435,-0.066900,-1.215864,...,0.084550,-0.637959,-0.797216,0.418649,0.160774,-0.561735,-0.802802,0.413062,0.158158,-0.564351
4,2021,105.0,26.0,-0.014800,-0.014005,-0.010380,-0.009585,0.078050,0.079506,1.252343,...,-2.446772,-0.155702,0.936597,-0.315746,-0.826929,1.464142,0.954071,-0.298272,-0.786689,1.504381


In [3021]:
SG_esg_int = SG_esg_int[['year', 'no of firms', 'no of firms(bottom 20)',
                            'rm_r_y1', 'rm_r_y2', 'eq_wg_ret_p', 'fc_wg_ret_p', 'lo_eq_wg_r_p',
                            'lo_eq_wg_r_p_y1', 'eq_r_alpha_y1', 'lo_eq_wg_r_p_y2', 'eq_r_alpha_y2', 'lo_fc_wg_r_p',
                            'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'lo_fc_wg_r_p_y2', 'fc_r_alpha_y2', 'lo_eq_wg_rc_p',
                            'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2', 'eq_rc_alpha_y2', 'lo_fc_wg_rc_p',
                            'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'lo_fc_wg_rc_p_y2', 'fc_rc_alpha_y2']]
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,2017,105.0,24.0,1.191128,2.113208,NaN,NaN,NaN,NaN,NaN,...,0.054252,0.651021,-0.540107,0.885089,-1.228119,0.055606,0.667277,-0.523851,0.914363,-1.198845
1,2018,105.0,25.0,-1.821289,-0.861280,-0.040609,-0.043508,-0.060739,-0.728871,1.092419,...,-0.061226,-0.734709,1.086580,-0.531474,0.329806,-0.064504,-0.774042,1.047247,-0.550732,0.310549
2,2019,105.0,25.0,-0.075984,-0.073393,0.020534,0.027175,0.004627,0.055522,0.131507,...,0.012872,0.154461,0.230445,0.165879,0.239272,0.012861,0.154331,0.230315,0.165729,0.239123
3,2020,105.0,25.0,-1.215864,-0.722509,-0.079245,-0.078131,-0.082295,-0.987538,0.228327,...,-0.066435,-0.797216,0.418649,-0.561735,0.160774,-0.066900,-0.802802,0.413062,-0.564351,0.158158
4,2021,105.0,26.0,1.252343,2.291070,-0.010380,-0.009585,-0.014800,-0.177594,-1.429937,...,0.078050,0.936597,-0.315746,1.464142,-0.826929,0.079506,0.954071,-0.298272,1.504381,-0.786689


## Average of the int_reporting

In [3022]:
SG_esg_int_mean = SG_esg_int.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
# SG_esg_int_mean.insert(0, 'year', 'Average', True)
# SG_esg_int_mean = pd.concat([SG_esg_int, SG_esg_int_mean], axis=0)
# SG_esg_int_mean.tail()

In [3023]:
SG_esg_int_mean['year'] = SG_esg_int_mean['year'].replace([SG_esg_int_mean['year'].mean()], 'average')
SG_esg_int_mean

,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,average,105.0,25.285714,-0.133933,0.549419,-0.020837,-0.020146,-0.038302,-0.45962,0.005579,...,0.003503,0.042031,0.175964,0.28438,-0.265039,0.003314,0.039767,0.1737,0.293878,-0.255541


## T-test of the int_reporting

a: an array of sample observations <br>
popmean: the expected population mean (The population mean is an average of a group characteristic.) <br>

In [3024]:
SG_res_dict = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, SG_esg_int.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=SG_esg_int.dropna()[SG_esg_int.columns[column]], popmean=0)
    temp_dict = {SG_esg_int.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict = {**SG_res_dict, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict[i]
                                    for i in SG_res_dict.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict

,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y2,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,t-score,-0.686899,0.216696,-1.286907,-1.151742,-1.908326,-1.908326,0.010626,-1.974725,-0.723125,...,-0.268116,-0.268116,1.226830,0.283072,-0.089961,-0.278438,-0.278438,1.253950,0.285455,-0.076549
1,p-value,0.541480,0.842345,0.288434,0.332885,0.152377,0.152377,0.992189,0.142788,0.521883,...,0.805989,0.805989,0.307388,0.795531,0.933988,0.798765,0.798765,0.298677,0.793870,0.943801


In [3025]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df])
SG_esg_int.tail()

,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,average,105.0,25.285714,-0.133933,0.549419,-0.020837,-0.020146,-0.038302,-0.459620,0.005579,...,0.003503,0.042031,0.175964,0.284380,-0.265039,0.003314,0.039767,0.173700,0.293878,-0.255541
0,t-score,NaN,NaN,-0.686899,0.216696,-1.286907,-1.151742,-1.908326,-1.908326,0.010626,...,-0.268116,-0.268116,1.226830,0.283072,-0.089961,-0.278438,-0.278438,1.253950,0.285455,-0.076549
1,p-value,NaN,NaN,0.541480,0.842345,0.288434,0.332885,0.152377,0.152377,0.992189,...,0.805989,0.805989,0.307388,0.795531,0.933988,0.798765,0.798765,0.298677,0.793870,0.943801


### Export the result into excel file

In [3026]:
sort_companies_data_bottom20per.to_excel(bottom_path, sheet_name="main", index=False)
SG_esg_int.to_excel(reporting_path, sheet_name="main", index=False)
